## Import Libraries 

In [1]:
import os 
import yaml 
import random 
import pickle
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

from tqdm import tqdm 

random.seed( 17 )

In [2]:
plt.rcParams['font.size'] = 14 
plt.rcParams['figure.figsize'] = (15,5)
plt.rcParams['lines.linewidth'] = 2

In [3]:
# parent dir 
parent_dir = '/data/NSE/bindata_indices/'
# f1 : 20250301 
# f2 : 01-03-2025
avl_datesf1 = [ name for name in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, name)) and name.startswith('20')]
avl_datesf2 = pd.to_datetime( avl_datesf1 , format='%Y%m%d')
dates = avl_datesf2.strftime('%d-%m-%Y')
print( f'Number of Dates: {len(avl_datesf2)}')

Number of Dates: 354


In [4]:
format1 = '%Y%m%d'
format2 = '%d-%m-%Y %H:%M:%S'

In [5]:
with open('user_input.yaml', 'r') as f:
    inputs = yaml.safe_load(f)
inputs 

{'underlying': 'NSEFNO_BANKNIFTY',
 'exp': 'H25',
 'start_date': '20250315',
 'end_date': '20250319',
 'dt': 5,
 'parent_dir': '/data/NSE/bindata_indices/'}

In [6]:
def drop_weekends(date_range: pd.DatetimeIndex) -> pd.DatetimeIndex:
    """Remove weekends (Saturday and Sunday) from a pandas date range."""
    return date_range[~date_range.weekday.isin([5, 6])]

In [7]:
# user input : date range 

datesf1 = pd.date_range( start= inputs['start_date'] , end = inputs['end_date'] , freq = 'D' )
datesf1 = drop_weekends( datesf1 )
datesf1 = datesf1.strftime( date_format= format1 ).to_list()

datesf1 

['20250317', '20250318', '20250319']

In [8]:
def format_dates(date_list):
    parsed_dates = pd.to_datetime(date_list, format='%Y%m%d')
    return parsed_dates.strftime('%d-%m-%Y').tolist()

In [9]:
datesf2 = format_dates( datesf1 )
datesf2

['17-03-2025', '18-03-2025', '19-03-2025']

In [10]:
def get_log_file_path( date_name : str , parent_dir : str =  '/data/NSE/bindata_indices/' ) : 
    dir = os.path.join( parent_dir , date_name , 'bin_data_archival_' + date_name + '.log')
    return dir 

In [11]:
dates_log_path = [ get_log_file_path( date ) for date in datesf1 ]
dates_log_path

['/data/NSE/bindata_indices/20250317/bin_data_archival_20250317.log',
 '/data/NSE/bindata_indices/20250318/bin_data_archival_20250318.log',
 '/data/NSE/bindata_indices/20250319/bin_data_archival_20250319.log']

In [12]:
for path , date in zip( dates_log_path , datesf2 ) : 
    small_chunk_data = pd.read_csv( path  ,  nrows  = 20  , usecols= [0,1,2,4,8,12] )
    print(f'Log file for {date}')
    print( small_chunk_data )
    print('-'*100)

Log file for 17-03-2025
        Date      Time                                      Symbol  \
0   20250317  09:15:01                        NSEFNO_BANKNIFTY_H25   
1   20250317  09:15:01  NSEFNO_BANKNIFTY_H25_40500.00_BANKNIFTY_PE   
2   20250317  09:15:01  NSEFNO_BANKNIFTY_H25_41000.00_BANKNIFTY_PE   
3   20250317  09:15:01  NSEFNO_BANKNIFTY_H25_41500.00_BANKNIFTY_PE   
4   20250317  09:15:01  NSEFNO_BANKNIFTY_H25_42000.00_BANKNIFTY_PE   
5   20250317  09:15:01  NSEFNO_BANKNIFTY_H25_42500.00_BANKNIFTY_PE   
6   20250317  09:15:01  NSEFNO_BANKNIFTY_H25_43000.00_BANKNIFTY_PE   
7   20250317  09:15:01  NSEFNO_BANKNIFTY_H25_43100.00_BANKNIFTY_PE   
8   20250317  09:15:01  NSEFNO_BANKNIFTY_H25_43200.00_BANKNIFTY_PE   
9   20250317  09:15:01  NSEFNO_BANKNIFTY_H25_43500.00_BANKNIFTY_PE   
10  20250317  09:15:01  NSEFNO_BANKNIFTY_H25_44000.00_BANKNIFTY_PE   
11  20250317  09:15:01  NSEFNO_BANKNIFTY_H25_44100.00_BANKNIFTY_PE   
12  20250317  09:15:01  NSEFNO_BANKNIFTY_H25_44200.00_BANKNIFTY_PE

In [13]:
import datetime 
import calendar 
import holidays 

### User Inputs : 

In [14]:
from datetime import time 

### Find the Strike Gap for each Day: 

In [15]:
# strike gap : 
st_gap = []
symbol = inputs['underlying'] + '_' + inputs['exp']
bar = tqdm(zip( datesf2 , dates_log_path ) , total = len(datesf1) , ncols = 125  )

for date , path in bar :
    bar.set_postfix_str(f'Processing Strike Gap for {date}')  
    find_16 = False 
    temp_df = []
    
    for df in pd.read_csv(dates_log_path[0] , chunksize=10000 , usecols = [1,2,8] ) : 
        df.Time = pd.to_datetime( df.Time , format = "%H:%M:%S" ).dt.time 
        df = df.set_index("Time")
        df = df[df.index == time( 9 , 16 , 1 )]
        
        if not df.empty : 
            if not find_16 : 
                find_16 = True 
            # keep only options symbols 
            df = df[
                (df.Symbol.str.startswith(symbol) )& 
                (df.Symbol != symbol) 
            ] 
            df.loc[: , 'Strike' ] = df.Symbol.str.split('_' , expand = True )[3].astype(float)
            
            temp_df.append( df.copy() )
        else : 
            if find_16 : 
                # tqdm.write('Completed Finding all the strike prices')
                # tqdm.write('-'*30)
                temp_df = pd.concat( temp_df )
                unique , counts = np.unique(np.diff(temp_df.Strike.unique()) , return_counts = True )
                st_gap.append( unique[counts == counts.max()].min() )
                break 
            else : 
                continue 

100%|████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.46it/s, Processing Strike Gap for 19-03-2025]


In [16]:
st_gap

[np.float64(100.0), np.float64(100.0), np.float64(100.0)]

## Make the Required DataFrame : 

In [17]:
# helper functions : 

def fill_comn( df1 , df2 , col1 , col2 ) : 
    common_idx = df1.index.intersection(df2.index)
    df1.loc[common_idx , col1 ] = df2.loc[ common_idx , col2 ]

def init_df(date) :
    df = pd.DataFrame() 

    df.index = pd.date_range(
        start =  date +  ' ' + "09:16:01" , 
        end = date  + ' ' + "15:30:01" , 
        freq= f"{inputs['dt']}min"
    )

    cols = ['Spot', 'ATM_Strike' , 'PE', 'CE']
    df[cols] = np.nan

    return df 

def get_file_name(date , inputs ) : 
    return f"cache/{date}_{inputs['underlying']}_{inputs['exp']}_dt_{inputs['dt']}.pickel"

In [18]:
def last_thursday(year, month):
    # Find the last day of the month
    last_day = calendar.monthrange(year, month)[1]
    # Create a date object for the last day of the month
    last_date = datetime.date(year, month, last_day)
    # Calculate the offset to the last Thursday (weekday 3)
    offset = (last_date.weekday() - 3) % 7
    # Subtract the offset to get the last Thursday
    last_thursday_date = last_date - datetime.timedelta(days=offset)
    return last_thursday_date


nse_code_to_month = {
    'F': 1,
    'G': 2,
    'H': 3,
    'J': 4,
    'K': 5,
    'M': 6,
    'N': 7,
    'Q': 8,
    'U': 9,
    'V': 10,
    'X': 11,
    'Z': 12
}


india_holidays  = holidays.India()

def get_exp_date(code):
    month = nse_code_to_month[code[0]]
    year = 2000 + int(code[1:])
    date = last_thursday( year , month )

    while date in india_holidays : 
        date  = date - datetime.timedelta(days=1)

    return date 

exp_date = get_exp_date(inputs['exp'])
exp_date

datetime.date(2025, 3, 27)

In [19]:
( exp_date - pd.to_datetime( datesf1[0] ).date() ).days + 1 

11

In [20]:
def create_df( date , path , gap ) : 
    # start reading the data in chunks : 
    for chunk in pd.read_csv(path, chunksize=100000, usecols = [1,2,8] ) :
        # convert the time to index : 
        chunk.Time = pd.to_datetime( date + ' ' + chunk.Time  )
        chunk = chunk.set_index('Time')
        chunk.Close = chunk.Close/100 

        # check if the time indices exist for the current chunk : 
        if chunk[ chunk.index.isin(df.index) ].empty: 
            continue 
        
        # Identify matching indices
        future_chunk = chunk[ chunk.Symbol == symbol ].copy() 
        
        # fill Spot values : 
        fill_comn( df , future_chunk , 'Spot' , 'Close')
        
        # get the ATM prices 
        future_chunk['ATM'] = (future_chunk['Close']/gap).round(0)*gap
        fill_comn( df , future_chunk , 'ATM_Strike' , 'ATM')
        
        # GET the ATM PUT and CALL close prices : 
        chunk  = chunk[
            chunk['Symbol'].str.startswith(symbol) & 
            (chunk['Symbol'] != symbol ) 
        ]
        split_cols = chunk['Symbol'].str.split('_', expand=True)
        chunk.loc[:, 'Strike'] = split_cols[3].astype(float)
        chunk.loc[:, 'Type'] = split_cols[5]
        
        df_reset = df.reset_index()
        chunk.reset_index( inplace = True )

        # Put : 
        matched = pd.merge(
            df_reset, 
            chunk, 
            left_on=['index', 'ATM_Strike'], 
            right_on=['Time', 'Strike'], 
            how='inner'
        ).set_index('index')
        fill_comn( df , matched[matched.Type == 'PE'] , 'PE' , 'Close' )
        fill_comn( df , matched[matched.Type == 'CE'] , 'CE' , 'Close' )
        df['t'] = (( exp_date - pd.to_datetime( date ).date() ).days + 1 )/365
    return df 

In [21]:
# Complete Creating the DataFrame : 

df_list = []
bar = tqdm(zip(datesf1 , dates_log_path , st_gap ) , total = len( datesf1) , ncols = 125 )

for date, path, gap in bar : 
    file_name = get_file_name( date , inputs )

    # check if the corresponding picke file exists, if not create it : 
    try : 
        with open( file_name , 'rb' ) as f : 
            df = pickle.load( f )
    except : 
        # init the data frame : 
        df = init_df(date)
        bar.set_postfix_str(f'Creating {file_name}. This is a one time process .....')
        df = create_df( date , path , gap )
        with open( file_name , 'wb' ) as f : 
            pickle.dump( df , f )
    df_list.append( df )
    
main_df = pd.concat( df_list)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 129.10it/s]


In [22]:
main_df.isna().any().any()

np.False_

In [23]:
main_df

,Spot,ATM_Strike,PE,CE,t
2025-03-17 09:16:01,48328.40,48300.0,482.22,494.01,0.030137
2025-03-17 09:21:01,48424.52,48400.0,478.06,490.25,0.030137
2025-03-17 09:26:01,48446.62,48400.0,457.75,501.82,0.030137
2025-03-17 09:31:01,48505.08,48500.0,477.73,464.73,0.030137
2025-03-17 09:36:01,48469.23,48500.0,490.53,455.73,0.030137
...,...,...,...,...,...
2025-03-19 15:06:01,49776.06,49800.0,411.75,374.41,0.024658
2025-03-19 15:11:01,49805.05,49800.0,401.20,390.45,0.024658
2025-03-19 15:16:01,49803.84,49800.0,407.57,394.41,0.024658
2025-03-19 15:21:01,49804.80,49800.0,405.63,394.85,0.024658


### Calculate Implied Vol using Spot Future Price and no Implied Vol

here we assume the future spot price to include the effect of intrest rate. Effective intrest rate = 0. 

In [24]:
import py_vollib_vectorized as pvl 

In [25]:
# for call options at atm using the balck scholes model: 
main_df['CE_IV_BS'] = pvl.implied_volatility.vectorized_implied_volatility(
    price = main_df['CE'] , 
    S = main_df['Spot'] , 
    K = main_df['ATM_Strike'] , 
    t = main_df['t'] , 
    r = 0 , 
    flag = 'c' , 
    on_error = 'warn' , 
    return_as = 'array'
) *100 

# same for put options : 
main_df['PE_IV_BS'] = pvl.implied_volatility.vectorized_implied_volatility(
    price = main_df['PE'] , 
    S = main_df['Spot'] , 
    K = main_df['ATM_Strike'] , 
    t = main_df['t'] , 
    r = 0 , 
    flag = 'p' , 
    on_error = 'warn' , 
    return_as = 'array'
) * 100 

In [26]:
# for call options using the black 76 model : 
main_df['CE_IV_B76'] = pvl.implied_volatility.vectorized_implied_volatility_black(
    price = main_df['CE'] , 
    F = main_df['Spot'] , 
    K = main_df['ATM_Strike'] , 
    t = main_df['t'] , 
    r = 6.5/100 , 
    flag = 'c' , 
    on_error = 'warn' , 
    return_as = 'array'
) * 100 


# for put options using the black 76 model ; 
main_df['PE_IV_B76'] = pvl.implied_volatility.vectorized_implied_volatility_black(
    price = main_df['PE'] , 
    F = main_df['Spot'] , 
    K = main_df['ATM_Strike'] , 
    t = main_df['t'] , 
    r = 6.5/100 , 
    flag = 'p' , 
    on_error = 'warn' , 
    return_as = 'array'
) * 100 

In [27]:
main_df

,Spot,ATM_Strike,PE,CE,t,CE_IV_BS,PE_IV_BS,CE_IV_B76,PE_IV_B76
2025-03-17 09:16:01,48328.40,48300.0,482.22,494.01,0.030137,14.335889,14.832466,14.364848,14.860734
2025-03-17 09:21:01,48424.52,48400.0,478.06,490.25,0.030137,14.253568,14.621418,14.282247,14.649384
2025-03-17 09:26:01,48446.62,48400.0,457.75,501.82,0.030137,14.258023,14.334123,14.287388,14.360909
2025-03-17 09:31:01,48505.08,48500.0,477.73,464.73,0.030137,13.759487,14.297769,13.786617,14.325658
2025-03-17 09:36:01,48469.23,48500.0,490.53,455.73,0.030137,14.025755,14.145821,14.052378,14.174477
...,...,...,...,...,...,...,...,...,...
2025-03-19 15:06:01,49776.06,49800.0,411.75,374.41,0.024658,12.384513,12.814291,12.403775,12.835474
2025-03-19 15:11:01,49805.05,49800.0,401.20,390.45,0.024658,12.434049,12.940512,12.454124,12.961140
2025-03-19 15:16:01,49803.84,49800.0,407.57,394.41,0.024658,12.580601,13.125534,12.600880,13.146490
2025-03-19 15:21:01,49804.80,49800.0,405.63,394.85,0.024658,12.579143,13.078556,12.599445,13.099412


In [29]:
# intrest rate : 
r = 6.5 /100

### Using Discounted Future Prices 

In [30]:
### Using Discounted future Price with intrest free rate using the black scholes model :

# discounted future prices :  
main_df['Discount_Spot'] = main_df['Spot'] * np.exp( -main_df['t']*r )

# call options IV : 
main_df['CE_IV_BS_Using_Discount'] = pvl.implied_volatility.vectorized_implied_volatility(
    price = main_df['CE'] , 
    S = main_df['Discount_Spot'] , 
    K = main_df['ATM_Strike'] , 
    t = main_df['t'] , 
    r = r , 
    flag = 'c' , 
    on_error='warn' , 
    return_as = 'array' 
) * 100 

# For Put options : 
main_df['PE_IV_BS_Using_Discount'] = pvl.implied_volatility.vectorized_implied_volatility(
    price = main_df['PE'] , 
    S = main_df['Discount_Spot'] , 
    K = main_df['ATM_Strike'] , 
    t = main_df['t'] , 
    r = r , 
    flag = 'p' , 
    on_error='warn' , 
    return_as = 'array' 
) * 100 

In [31]:
main_df

,Spot,ATM_Strike,PE,CE,t,CE_IV_BS,PE_IV_BS,CE_IV_B76,PE_IV_B76,Discount_Spot,CE_IV_BS_Using_Discount,PE_IV_BS_Using_Discount
2025-03-17 09:16:01,48328.40,48300.0,482.22,494.01,0.030137,14.335889,14.832466,14.364848,14.860734,48233.821964,14.364848,14.860734
2025-03-17 09:21:01,48424.52,48400.0,478.06,490.25,0.030137,14.253568,14.621418,14.282247,14.649384,48329.753858,14.282247,14.649384
2025-03-17 09:26:01,48446.62,48400.0,457.75,501.82,0.030137,14.258023,14.334123,14.287388,14.360909,48351.810609,14.287388,14.360909
2025-03-17 09:31:01,48505.08,48500.0,477.73,464.73,0.030137,13.759487,14.297769,13.786617,14.325658,48410.156203,13.786617,14.325658
2025-03-17 09:36:01,48469.23,48500.0,490.53,455.73,0.030137,14.025755,14.145821,14.052378,14.174477,48374.376361,14.052378,14.174477
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-19 15:06:01,49776.06,49800.0,411.75,374.41,0.024658,12.384513,12.814291,12.403775,12.835474,49696.345829,12.403775,12.835474
2025-03-19 15:11:01,49805.05,49800.0,401.20,390.45,0.024658,12.434049,12.940512,12.454124,12.961140,49725.289403,12.454124,12.961140
2025-03-19 15:16:01,49803.84,49800.0,407.57,394.41,0.024658,12.580601,13.125534,12.600880,13.146490,49724.081340,12.600880,13.146490
2025-03-19 15:21:01,49804.80,49800.0,405.63,394.85,0.024658,12.579143,13.078556,12.599445,13.099412,49725.039803,12.599445,13.099412


In [32]:
(main_df['CE_IV_BS_Using_Discount'] != main_df['CE_IV_B76']).sum()
# CE using discount and BS is same as CE using Black 76 

np.int64(0)

In [33]:
(main_df['PE_IV_BS_Using_Discount'] != main_df['PE_IV_B76']).sum()
# PE using discount and BS is same as CE using Black 76 

np.int64(0)

In [35]:
# differenec in vol values for call : 
main_df['CE_Diff'] = main_df['CE_IV_BS_Using_Discount'] - main_df['CE_IV_BS']
# difference in vol values for puts : 
main_df['PE_Diff'] = main_df['PE_IV_BS_Using_Discount'] - main_df['PE_IV_BS']

main_df.describe().to_excel('Tables/summary.xlsx')
main_df.describe()

,Spot,ATM_Strike,PE,CE,t,CE_IV_BS,PE_IV_BS,CE_IV_B76,PE_IV_B76,Discount_Spot,CE_IV_BS_Using_Discount,PE_IV_BS_Using_Discount,CE_Diff,PE_Diff
count,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000
mean,49117.930267,49117.777778,440.230133,427.710178,0.027397,13.185949,13.577265,13.209521,13.601540,49030.614300,13.209521,13.601540,0.023572,0.024275
std,534.116563,529.777956,31.707704,28.251865,0.002242,0.443348,0.438231,0.445695,0.440719,540.185698,0.445695,0.440719,0.002631,0.002901
min,48328.400000,48300.000000,382.210000,366.490000,0.024658,12.384513,12.814291,12.403775,12.835474,48233.821964,12.403775,12.835474,0.018840,0.019660
25%,48498.410000,48500.000000,414.300000,406.110000,0.024658,12.759233,13.177302,12.783163,13.200753,48403.499256,12.783163,13.200753,0.021351,0.021517
50%,49087.870000,49100.000000,441.270000,425.520000,0.027397,13.231026,13.611341,13.255383,13.636097,49000.531036,13.255383,13.636097,0.023394,0.024195
75%,49737.350000,49700.000000,467.250000,447.130000,0.030137,13.517439,13.962998,13.543060,13.989193,49657.697821,13.543060,13.989193,0.025717,0.026907
max,49874.910000,49900.000000,498.170000,501.820000,0.030137,14.335889,14.832466,14.364848,14.860734,49795.037525,14.364848,14.860734,0.029365,0.029111


In [36]:
main_df.to_excel('Tables/main_df.xlsx')
main_df

,Spot,ATM_Strike,PE,CE,t,CE_IV_BS,PE_IV_BS,CE_IV_B76,PE_IV_B76,Discount_Spot,CE_IV_BS_Using_Discount,PE_IV_BS_Using_Discount,CE_Diff,PE_Diff
2025-03-17 09:16:01,48328.40,48300.0,482.22,494.01,0.030137,14.335889,14.832466,14.364848,14.860734,48233.821964,14.364848,14.860734,0.028960,0.028268
2025-03-17 09:21:01,48424.52,48400.0,478.06,490.25,0.030137,14.253568,14.621418,14.282247,14.649384,48329.753858,14.282247,14.649384,0.028679,0.027966
2025-03-17 09:26:01,48446.62,48400.0,457.75,501.82,0.030137,14.258023,14.334123,14.287388,14.360909,48351.810609,14.287388,14.360909,0.029365,0.026786
2025-03-17 09:31:01,48505.08,48500.0,477.73,464.73,0.030137,13.759487,14.297769,13.786617,14.325658,48410.156203,13.786617,14.325658,0.027130,0.027889
2025-03-17 09:36:01,48469.23,48500.0,490.53,455.73,0.030137,14.025755,14.145821,14.052378,14.174477,48374.376361,14.052378,14.174477,0.026623,0.028656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-19 15:06:01,49776.06,49800.0,411.75,374.41,0.024658,12.384513,12.814291,12.403775,12.835474,49696.345829,12.403775,12.835474,0.019262,0.021183
2025-03-19 15:11:01,49805.05,49800.0,401.20,390.45,0.024658,12.434049,12.940512,12.454124,12.961140,49725.289403,12.454124,12.961140,0.020076,0.020628
2025-03-19 15:16:01,49803.84,49800.0,407.57,394.41,0.024658,12.580601,13.125534,12.600880,13.146490,49724.081340,12.600880,13.146490,0.020279,0.020956
2025-03-19 15:21:01,49804.80,49800.0,405.63,394.85,0.024658,12.579143,13.078556,12.599445,13.099412,49725.039803,12.599445,13.099412,0.020302,0.020856


In [39]:
for i in range(4) : 
    print( main_df.iloc[i , : ])
    print('-'*30)

Spot                       48328.400000
ATM_Strike                 48300.000000
PE                           482.220000
CE                           494.010000
t                              0.030137
CE_IV_BS                      14.335889
PE_IV_BS                      14.832466
CE_IV_B76                     14.364848
PE_IV_B76                     14.860734
Discount_Spot              48233.821964
CE_IV_BS_Using_Discount       14.364848
PE_IV_BS_Using_Discount       14.860734
CE_Diff                        0.028960
PE_Diff                        0.028268
Name: 2025-03-17 09:16:01, dtype: float64
------------------------------
Spot                       48424.520000
ATM_Strike                 48400.000000
PE                           478.060000
CE                           490.250000
t                              0.030137
CE_IV_BS                      14.253568
PE_IV_BS                      14.621418
CE_IV_B76                     14.282247
PE_IV_B76                     14.649384
Discoun